In [1]:
import os
import sys
import numpy as np
import scipy.io as io
import scipy.ndimage.filters as filters

In [1049]:
import BVH
import Animation
from Quaternions import Quaternions
from Pivots import Pivots

In [1254]:
def process_file(filename, window=240, window_step=120, export_trajectory=False):
    
    anim, names, frametime = BVH.load(filename)
    
    """ Subsample to 60 fps """
    anim = anim[::2]
    
    """ Do FK """
    global_xforms = Animation.transforms_global(anim)  # intermediate
    global_positions = global_xforms[:,:,:3,3] / global_xforms[:,:,3:,3]
    global_rotations = Quaternions.from_transforms(global_xforms)
    

    """ Remove Uneeded Joints """ #>># done post-hoc in PFNN
    used_joints = np.array([
         0,
         2,  3,  4,  5,
         7,  8,  9, 10,
        12, 13, 15, 16,
        18, 19, 20, 22,
        25, 26, 27, 29])
         
    positions = global_positions[:,used_joints]
    global_rotations = global_rotations[:,used_joints]
    # ________________________________________________________

    """ Put on Floor """
    positions[:,:,1] -= positions[:,:,1].min()
    
    """ Get Foot Contacts """
    if True:
        velfactor, heightfactor = np.array([0.05,0.05]), np.array([3.0, 2.0])
        
        fid_l, fid_r = np.array([3,4]), np.array([7,8])
        feet_l_x = (positions[1:,fid_l,0] - positions[:-1,fid_l,0])**2
        feet_l_y = (positions[1:,fid_l,1] - positions[:-1,fid_l,1])**2
        feet_l_z = (positions[1:,fid_l,2] - positions[:-1,fid_l,2])**2
        feet_l_h = positions[:-1,fid_l,1]
        feet_l = (((feet_l_x + feet_l_y + feet_l_z) < velfactor) & (feet_l_h < heightfactor)).astype(np.float)
        
        feet_r_x = (positions[1:,fid_r,0] - positions[:-1,fid_r,0])**2
        feet_r_y = (positions[1:,fid_r,1] - positions[:-1,fid_r,1])**2
        feet_r_z = (positions[1:,fid_r,2] - positions[:-1,fid_r,2])**2
        feet_r_h = positions[:-1,fid_r,1]
        feet_r = (((feet_r_x + feet_r_y + feet_r_z) < velfactor) & (feet_r_h < heightfactor)).astype(np.float)
    
    """ Get Root Velocity """
    velocity = (positions[1:] - positions[:-1]).copy()
    
    """ Remove Translation """
    positions[:,:,0] = positions[:,:,0] - positions[:,0:1,0]
    positions[:,:,2] = positions[:,:,2] - positions[:,0:1,2]
    
    """ Get Forward Direction """
    sdr_l, sdr_r, hip_l, hip_r = 13, 17, 1, 5
    across1 = positions[:,hip_l] - positions[:,hip_r]
    across0 = positions[:,sdr_l] - positions[:,sdr_r]
    across = across0 + across1
    across = across / np.sqrt((across**2).sum(axis=-1))[...,np.newaxis]
    
    direction_filterwidth = 20
    forward = np.cross(across, np.array([[0,1,0]]))
    forward = filters.gaussian_filter1d(forward, direction_filterwidth, axis=0, mode='nearest')    
    forward = forward / np.sqrt((forward**2).sum(axis=-1))[...,np.newaxis]
    
    """ Get Root Rotation """
    target = np.array([[0,0,1]]).repeat(len(forward), axis=0)
    root_rotation = Quaternions.between(forward, target)[:,np.newaxis]   # rotation needed fwd->z? 
    rvelocity = (root_rotation[1:] * -root_rotation[:-1]).to_pivots()
    
    """ Local Space """  # NEW: define position of joints relative to 
    local_positions = positions.copy()
    local_positions[:,:,0] = local_positions[:,:,0] - local_positions[:,0:1,0]  # x rel to root x
    local_positions[:,:,2] = local_positions[:,:,2] - local_positions[:,0:1,2]  # z rel to root z
    
    local_positions = root_rotation[:-1] * local_positions[:-1]   # remove Y rotation from pos
#     local_velocities = root_rotation[:-1] * velocity              # remove Y rotation from vel
    local_velocities = local_positions[1:] - local_positions[:-1]
    local_rotations = abs((root_rotation[:-1] * global_rotations[:-1])).log()

    root_rvelocity = Pivots.from_quaternions(root_rotation[1:] * -root_rotation[:-1]).ps

    # ================================================================
    #                Construct Model Matrices
    # ================================================================
    Xc, Yc = [], []
    window = 60
    ixs = range(window, len(anim)-window-1, 1)
    
    """ Construct X Vectors (Inputs) """
    for i in ixs:
        cur_rot = root_rotation[i:i+1,0]
        cur_pos = global_positions[i:i+1,0]  # root joint
        rootposs = cur_rot * (global_positions[i-window:i+window:10,0] - cur_pos)
        rootdirs = cur_rot * forward[i-window:i+window:10]
        
        Xc.append(np.hstack([
                rootposs[:,0].ravel(), rootposs[:,2].ravel(), # Trajectory Pos
                rootdirs[:,0].ravel(), rootdirs[:,2].ravel(), # Trajectory Dir
                local_positions[i-1].ravel(),  # Cur Joint Pos (note -1 b.c. first el omitted)
                local_velocities[i-1].ravel(), # Cur Joint Vel (note -1 b.c. first el omitted)
                ]))

    """ Construct Y Vectors (Outputs) """
    for i in ixs:
        next_rot = root_rotation[i+1:i+2,0]
        next_pos = global_positions[i+1:i+2,0]  # root joint
        rootposs_next = next_rot * (global_positions[i+1:i+window+1:10,0] - next_pos)
        rootdirs_next = next_rot * forward[i+1:i+window+1:10]
        
        Yc.append(np.hstack([
                root_rvelocity[i].ravel(),   # rotational vel at root
                np.concatenate([feet_l[i], feet_r[i]], axis=-1), # feet contacts
                rootposs_next[:,0].ravel(), rootposs_next[:,2].ravel(), # Next Trajectory Pos
                rootdirs_next[:,0].ravel(), rootdirs_next[:,2].ravel(), # Next Trajectory Dir
                local_positions[i].ravel(),  # Next Joint Pos
                local_velocities[i].ravel(), # Next Joint Vel
                local_rotations[i].ravel()   # Next Joint Rot
                ]))
    
    return Xc, Yc

<div style="background-color:#F04040;">
    <br>
    <center><b><span style="font-size:36px">Loop over all files, adding an additional dimension at start of each </span></b></center><br><br>
</div>

In [ ]:
...

In [1255]:
X, Y = process_file(files[3])

# Convert to Float32 (reduce memory footprint)
X = [X[i].astype('float32') for i in range(len(X))]
Y = [Y[i].astype('float32') for i in range(len(Y))]

# Extract my favourite bits of Y:
Y = [Y[i][[0,1,2,3,4, *list(range(29,29+63)), *list(range(29+63*2,29+63*3))]] for i in range(len(Y))]

<div style="background-color:#F0F040;">
    <br>
    <center><b><span style="font-size:36px">Normalize X, Y (DONE, CAREFUL THAT CALCING FOR ALL INPUT SETS) </span></b></center><br><br>
</div>

In [ ]:
""" Calculate Mean and Std """
Xmean, Xstd = X.mean(axis=0), X.std(axis=0)
Ymean, Ystd = Y.mean(axis=0), Y.std(axis=0)

j = 21
w = ((60*2)//10)

""" pool standard deviation over relevant dimensions of X"""
Xstd[w*0:w* 1] = Xstd[w*0:w* 1].mean() # Trajectory Past Positions
Xstd[w*1:w* 2] = Xstd[w*1:w* 2].mean() # Trajectory Future Positions
Xstd[w*2:w* 3] = Xstd[w*2:w* 3].mean() # Trajectory Past Directions
Xstd[w*3:w* 4] = Xstd[w*3:w* 4].mean() # Trajectory Future Directions

def ix_sec(x): return w*4+j*3*x
Xstd[ix_sec(0):ix_sec(1)] = Xstd[ix_sec(0):ix_sec(1)].mean() / (0.1) # Pos
Xstd[ix_sec(1):ix_sec(2)] = Xstd[ix_sec(0):ix_sec(1)].mean() / (0.1) # Vel

""" pool standard deviation over relevant dimensions of Y"""
Ystd[0:1] = Ystd[0:1].mean() # Rotational Velocity
Ystd[1:5] = Ystd[1:5].mean() # Foot Contacts

Ystd[5+w*0:5+w*1] = Ystd[5+w*0:5+w*1].mean() # Trajectory Future Positions
Ystd[5+w*1:5+w*2] = Ystd[5+w*1:5+w*2].mean() # Trajectory Future Directions

def ix_sec(y): return 5+w*2+j*3*y
Ystd[ix_sec(0):ix_sec(1)] = Ystd[ix_sec(0):ix_sec(1)].mean() # Pos
Ystd[ix_sec(1):ix_sec(2)] = Ystd[ix_sec(1):ix_sec(2)].mean() # <-- Rot as removed # Vel
# Ystd[ix_sec(2):ix_sec(3)] = Ystd[ix_sec(2):ix_sec(3)].mean() # Rot

""" Save Mean / Std / Min / Max """

np.savez_compressed("Xmean.npz", Xmean=Xmean)
np.savez_compressed("Ymean.npz", Ymean=Ymean)
np.savez_compressed("Xstd.npz", Xstd=Xstd)
np.savez_compressed("Ystd.npz", Ystd=Ystd)

""" Normalize Data """

X = (X - Xmean) / Xstd
Y = (Y - Ymean) / Ystd

<div style="background-color:#F04040;">
    <br>
    <center><b><span style="font-size:36px">Extract Windows </span></b></center><br><br>
</div>

In [ ]:
...

<div style="background-color:#F04040;">
    <br>
    <center><b><span style="font-size:36px">Ensure we can reconstruct absolute position from above</span></b></center><br><br>
</div>

In [ ]:
...

<div style="background-color:#F04040;">
    <br>
    <center><b><span style="font-size:36px">ENSURE Float32 and save for Julia </span></b></center><br><br>
</div>

In [ ]:
...

In [ ]:
[plt.plot([Y_[i][ix] for i in range(len(Y_))]) for ix in range(6,66)]
plt.gca().axhline(0, linestyle=":")

In [12]:
import pandas as pd
cmu_loco = pd.read_csv("../cmu/cmu_locomotion_lkp.txt", delimiter="\t", header=None)[0]

In [1018]:
_anim, _names, _frametime = BVH.load(files[3])

In [ ]:
fig, axs = plt.subplots(3,10, figsize=(15,12))
for jj in range(10):
    yloclpos = Y[ii//2 + jj,:-7].reshape(21,3)
    axs[0,jj].scatter(yloclpos[:,0], yloclpos[:,1])
    axs[1,jj].scatter(local_positions[ii + jj][:,0], local_positions[ii + jj][:,1])
    axs[0,jj].set_xlim(-5,5); axs[1,jj].set_xlim(-5,5)
    axs[0,jj].set_aspect("equal"); axs[1,jj].set_aspect("equal")
    
    axs[2,jj].scatter(yloclpos[:,0], yloclpos[:,1])
    axs[2,jj].scatter(local_positions[ii + jj][:,0], local_positions[ii + jj][:,1])
    axs[2,jj].set_xlim(-5,5); axs[2,jj].set_aspect("equal")

ii +=10

In [95]:
for i, item in enumerate(files[0:4]):
    print('Database %s Processing %i of %i (%s)' % ("cmu", i, len(files), item))
    tmp=process_file(item);
    print("DONE")

Database cmu Processing 0 of 203 (cmu/02_01.bvh)
(171, 1)
(171, 2)
(171, 63)
DONE
Database cmu Processing 1 of 203 (cmu/02_02.bvh)
(148, 1)
(148, 2)
(148, 63)
DONE
Database cmu Processing 2 of 203 (cmu/02_03.bvh)
(86, 1)
(86, 2)
(86, 63)
DONE
Database cmu Processing 3 of 203 (cmu/05_01.bvh)
(298, 1)
(298, 2)
(298, 63)
DONE


In [35]:
path = ''
database = 'cmu'

files = [os.path.join(path+database,f + ".bvh") for f in cmu_loco]
files = list(filter(lambda x: os.path.isfile(x) and x != 'rest.bvh', files))

clips = []

for i, item in enumerate(files):
    print('Database %s Processing %i of %i (%s)' % (database, i, len(files), item))
    clips += process_file(item)

clips = np.array(clips)
np.savez_compressed('proc/data_'+database+'_loco', clips=clips)

Database cmu Processing 0 of 203 (cmu/02_01.bvh)
Database cmu Processing 1 of 203 (cmu/02_02.bvh)
Database cmu Processing 2 of 203 (cmu/02_03.bvh)
Database cmu Processing 3 of 203 (cmu/05_01.bvh)
Database cmu Processing 4 of 203 (cmu/06_01.bvh)
Database cmu Processing 5 of 203 (cmu/07_01.bvh)
Database cmu Processing 6 of 203 (cmu/07_02.bvh)
Database cmu Processing 7 of 203 (cmu/07_03.bvh)
Database cmu Processing 8 of 203 (cmu/07_04.bvh)
Database cmu Processing 9 of 203 (cmu/07_05.bvh)
Database cmu Processing 10 of 203 (cmu/07_06.bvh)
Database cmu Processing 11 of 203 (cmu/07_07.bvh)
Database cmu Processing 12 of 203 (cmu/07_08.bvh)
Database cmu Processing 13 of 203 (cmu/07_09.bvh)
Database cmu Processing 14 of 203 (cmu/07_10.bvh)
Database cmu Processing 15 of 203 (cmu/07_11.bvh)
Database cmu Processing 16 of 203 (cmu/07_12.bvh)
Database cmu Processing 17 of 203 (cmu/08_01.bvh)
Database cmu Processing 18 of 203 (cmu/08_02.bvh)
Database cmu Processing 19 of 203 (cmu/08_03.bvh)
Database c

Database cmu Processing 163 of 203 (cmu/36_09.bvh)
Database cmu Processing 164 of 203 (cmu/37_01.bvh)
Database cmu Processing 165 of 203 (cmu/38_01.bvh)
Database cmu Processing 166 of 203 (cmu/38_02.bvh)
Database cmu Processing 167 of 203 (cmu/38_03.bvh)
Database cmu Processing 168 of 203 (cmu/38_04.bvh)
Database cmu Processing 169 of 203 (cmu/39_01.bvh)
Database cmu Processing 170 of 203 (cmu/39_02.bvh)
Database cmu Processing 171 of 203 (cmu/39_03.bvh)
Database cmu Processing 172 of 203 (cmu/39_04.bvh)
Database cmu Processing 173 of 203 (cmu/39_05.bvh)
Database cmu Processing 174 of 203 (cmu/39_06.bvh)
Database cmu Processing 175 of 203 (cmu/39_07.bvh)
Database cmu Processing 176 of 203 (cmu/39_08.bvh)
Database cmu Processing 177 of 203 (cmu/39_09.bvh)
Database cmu Processing 178 of 203 (cmu/39_10.bvh)
Database cmu Processing 179 of 203 (cmu/39_11.bvh)
Database cmu Processing 180 of 203 (cmu/39_12.bvh)
Database cmu Processing 181 of 203 (cmu/39_13.bvh)
Database cmu Processing 182 of 

In [467]:
len(clips_list)

1038

In [52]:
clips_list = clips

In [53]:
clips = np.array(clips)

In [297]:
np.arange(100)[2:10:4]

array([2, 6])

In [291]:
clips[0].shape

(240, 70)

## Visualisation / Animation

In [50]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import matplotlib.colors as colors
from matplotlib.animation import ArtistAnimation
import matplotlib.patheffects as pe

In [64]:
def animation_plot(animations, interval=33.33):
    
    footsteps = []
    
    for ai in range(len(animations)):
        anim = animations[ai][0].copy()
        
        joints, root_x, root_z, root_r = anim[:,:-7], anim[:,-7], anim[:,-6], anim[:,-5]
        
        joints = joints.reshape((len(joints), -1, 3))
        
        rotation = Quaternions.id(1)
        offsets = []
        translation = np.array([[0,0,0]])
        
        for i in range(len(joints)):
            joints[i,:,:] = rotation * joints[i]
            joints[i,:,0] = joints[i,:,0] + translation[0,0]
            joints[i,:,2] = joints[i,:,2] + translation[0,2]
            rotation = Quaternions.from_angle_axis(-root_r[i], np.array([0,1,0])) * rotation
            offsets.append(rotation * np.array([0,0,1]))
            translation = translation + rotation * np.array([root_x[i], 0, root_z[i]])
        
        animations[ai] = joints
        footsteps.append(anim[:,-4:])
        
    footsteps = np.array(footsteps)
    
    scale = 1.25*((len(animations))/2)
    
    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim3d(-scale*30, scale*30)
    ax.set_zlim3d( 0, scale*60)
    ax.set_ylim3d(-scale*30, scale*30)
    ax.set_xticks([], [])
    ax.set_yticks([], [])
    ax.set_zticks([], [])
    ax.set_aspect('equal')
    
    acolors = list(sorted(colors.cnames.keys()))[::-1]
    lines = []
    
    parents = np.array([-1,0,1,2,3,0,5,6,7,0,9,10,11,11,13,14,15,11,17,18,19])
    
    for ai, anim in enumerate(animations):
        lines.append([plt.plot([0,0], [0,0], [0,0], color=acolors[ai], 
            lw=2, path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])[0] for _ in range(anim.shape[1])])
    
    def animate(i):
        changed = []        
        for ai in range(len(animations)):
            offset = 25*(ai-((len(animations))/2))
            for j in range(len(parents)):
                if parents[j] != -1:
                    lines[ai][j].set_data(
                        [ animations[ai][i,j,0]+offset, animations[ai][i,parents[j],0]+offset],
                        [-animations[ai][i,j,2],       -animations[ai][i,parents[j],2]])
                    lines[ai][j].set_3d_properties(
                        [ animations[ai][i,j,1],        animations[ai][i,parents[j],1]])
            changed += lines
            
        return changed
        
    plt.tight_layout()
        
    ani = animation.FuncAnimation(fig, 
        animate, np.arange(len(animations[0])), interval=interval)
    
    
    ani.save('animation.mp4', writer=writer)
    plt.show()
        
    

In [56]:
database = clips

In [1021]:
database = [process_file(files[0]), process_file(files[3])]

In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

index0, index1 = 0, 1
animation_plot([
    database[index0:index0+1],
    database[index1:index1+1],
])

In [99]:
ii = 0